In [ ]:
import logging
import sys


date_strftime_format = "%Y-%m-%y %H:%M:%S"
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s", datefmt=date_strftime_format)

# Data

### Training data
- Source: http://mattmahoney.net/dc/text8.zip
- Stored in: `data/train.txt`

### Analogies data
- Source: https://raw.githubusercontent.com/nicholas-leonard/word2vec/refs/heads/master/questions-words.txt
- Stored in: `data/analogies.txt`

# Defining model

## Model wrapper

In [ ]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models.word2vec import Text8Corpus

In [ ]:
class StoreLossCurveCallback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.last_logged_loss = 0
        self.loss_curve = []

    def on_epoch_end(self, model):
        curr_loss = model.get_latest_training_loss() - self.last_logged_loss
        self.last_logged_loss = model.get_latest_training_loss()

        self.loss_curve.append(curr_loss)

        print(
            f"Loss for epoch #{self.epoch}: {curr_loss}"
        )

        self.epoch += 1

In [ ]:
class Word2VecModel:
    def __init__(
        self,
        model_type: str,
        window_size: int,
        embedding_size: int,
        min_word_count: int = 0,
        workers: int = 8,
    ):
        self.model_type = model_type
        self._sg = 1 if model_type ==  "skipgram" else 0
        self.window_size = window_size
        self.embedding_size = embedding_size
        self.min_word_count = min_word_count
        self.workers = workers
        self.compute_loss = True

        self._loss_container = StoreLossCurveCallback()
        self.loss_curve = []

        self.model =  Word2Vec(
            sg=self._sg,
            window=self.window_size,
            vector_size=self.embedding_size,
            min_count=self.min_word_count,
            workers=self.workers,
        )


    @property
    def wv(self):
        return self.model.wv


    def _build_vocab(self, training_corpus_fpath):
        self.model.build_vocab(
            corpus_iterable=Text8Corpus(fname=training_corpus_fpath),
        )

    def train(
        self,
        training_corpus_fpath: str,
        epochs: int,
        
    ):
        self._build_vocab(
            training_corpus_fpath=training_corpus_fpath
        )

        self.model.train(
            corpus_iterable=Text8Corpus(fname=training_corpus_fpath),
            total_examples=self.model.corpus_count,
            epochs=epochs,
            compute_loss=self.compute_loss,
            callbacks=[self._loss_container],
        )

        self.loss_curve = self._loss_container.loss_curve



## Searching best hyper-parameters configs

### Preparing test data

In [ ]:
analogies_file_name = "../data/analogies.txt"

with open(analogies_file_name) as file:
    file_content = file.read().splitlines()

all_test_analogies = {}
last_key_added = None
for line in file_content:
    if line[0] == ":":
        last_key_added = line.replace(": ", "")
        all_test_analogies[last_key_added] = []

    else:
        all_test_analogies[last_key_added].append(
            line.lower().split(" ")
        )

### Defining evaluation

In [ ]:
import numpy as np
from scipy.spatial import distance

In [ ]:
def evaluate_analogy(model, word_tuple):
    w0_embedding = model.wv[word_tuple[0]]
    w1_embedding = model.wv[word_tuple[1]]
    w2_embedding = model.wv[word_tuple[2]]
    w3_embedding = model.wv[word_tuple[3]]

    return 1 - distance.cosine(
        w1_embedding - w0_embedding,
        w3_embedding - w2_embedding,
    )

In [ ]:
def build_report(model, test_analogies):
    total_ignored_analogies = 0

    report = {k: 0.0 for k in test_analogies.keys()}
    for sub_category in report.keys():
        similarities = []
        for curr_sample in test_analogies[sub_category]:
            if all([model.wv.__contains__(sample) for sample in curr_sample]):
                curr_similarity = evaluate_analogy(model, curr_sample)
                similarities.append(curr_similarity)

            else:
                total_ignored_analogies += 1
        
        report[sub_category] = np.average(similarities)

    if total_ignored_analogies:
        print(
            f"[WARNING] A total of {total_ignored_analogies} samples were ignored because they contained "
            "words out of the model's vocabulary."
        )

    report["overall_average"] = np.average(list(report.values()))

    return report

### Grid search routine

In [ ]:
from itertools import product

In [ ]:
def run_grid_search(
    param_grid: dict,
    test_analogies: dict,
    param_conditions_callback: callable = None,
    return_best: bool = False
):
    param_combinations = list(product(*param_grid.values()))

    param_keys = list(param_grid.keys())

    curr_train = 0

    results = []
    for params in param_combinations:
        param_dict = dict(zip(param_keys, params))

        if param_conditions_callback and not param_conditions_callback(param_dict):
            continue

        model = Word2VecModel(
            model_type=param_dict["model_type"],
            window_size=param_dict["window_size"],
            embedding_size=param_dict["embedding_size"],
            workers=4,
        )

        print(
            f"Starting training model {curr_train + 1} of {len(param_combinations)}"
        )

        model.train(
            training_corpus_fpath="../data/train.txt",
            epochs=param_dict["epochs"],
        )

        curr_model_report = build_report(model, test_analogies)
        score = curr_model_report["overall_average"]

        print(
            f"model_type: {model.model_type}, window_size: {model.window_size}, embedding_size: {model.embedding_size}"
        )
        print(f"Final score: {score}\n")

        results.append(
            {"params": param_dict, "score": score, "full_report": curr_model_report, "model": model}
        )

        curr_train += 1

    if not return_best:
        return results

    return max(results, key=lambda x: x["score"])

In [ ]:
def is_valid_param_combination(selected_params: dict):
    return (
        selected_params["window_size"]
        <= selected_params["embedding_size"]
    )

In [ ]:
%%time

param_grid = {
    "model_type": ["cbow"],
    "embedding_size": [5, 20],
    "window_size": [3, 15],
    "epochs": [5, 50],
}

grid_search_results = run_grid_search(
    param_grid=param_grid,
    test_analogies=all_test_analogies,
    param_conditions_callback=is_valid_param_combination,
    return_best=False
)

grid_search_results

In [ ]:
import matplotlib.pyplot as plt

plt.plot(grid_search_results["model"].loss_curve)
plt.show()

In [ ]:
model = Word2VecModel(
    model_type=0,
    window_size=3,
    embedding_size=10,
)

model._build_vocab(
    training_corpus_fpath="../data/train.txt",
)

curr_model_report = build_report(model, all_test_analogies)
curr_model_report